### boston 집값 예측
target value: 실제 주택 가격

### NN regressor 모델 설계
- classification과 크게 코드차이가 없다.
- 차이:
1.   train셋과 test 셋 Scaler를 통해 transofrom을 해준다.
- 데이터의 분포가 다 다르기때문에 scaler를 맞춰주었다. preprocessing.StandardScaler() 를 사용.
2.   손실함수로 MSELoss를 사용한다.
3. 평가는 RMSE 




## baseline을 넘기 위해 내가 한 전략
- 1) 1layer 에서 3layer사용 -> 성능이 더 좋지 않음.
- 2) 3lyaer + dropout 적용-> **baseline을 넘겼다.**
- 3) 2번에서 13>256>1 을 13>512>1 로 변경했다. -> **성능이 조금 더 향상되었다.** 
- 4) 4layer + dropout -> cost가 전혀 낮아지지 않았다. 
- 5) 4layer + dropout + 파라미터변경 + hidden layer노드수 변경 -> cost가 전혀 낮아지지 않았다.

--->  3)번을 최종으로 제출. **8.73479**

In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.1.1)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 64 Jun 15 09:42 kaggle.json


In [ ]:
!kaggle competitions download -c 2020aiboston

  0% 0.00/14.5k [00:00<?, ?B/s]
100% 14.5k/14.5k [00:00<00:00, 11.9MB/s]


In [ ]:
!unzip 2020aiboston.zip

Archive:  2020aiboston.zip
  inflating: boston_housing_test.csv  
  inflating: boston_housing_train.csv  
  inflating: sample_submission.csv   


In [ ]:
train_data=pd.read_csv('boston_housing_train.csv',header=None, usecols=range(1,15))
test_data=pd.read_csv('boston_housing_test.csv',header=None, usecols=range(1,14))

In [1]:
import pandas as pd
import numpy as np

import torch
import torchvision.datasets as data
import torchvision.transforms as transforms
import random

from sklearn import preprocessing

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
# 학습 파라미터 설정
learning_rate = 0.1
training_epochs = 300
batch_size = 100

drop_prob = 0.3

Scaler = preprocessing.StandardScaler()  

# 데이터의 분포가 다 다르다. 이를 preprocessing.StandardScaler()를 이용해  Scaler를 맞춰주었다. 

In [ ]:
x_train_data=train_data.loc[:,0:13]
y_train_data=train_data.loc[:,14]

x_train_data=np.array(x_train_data)
y_train_data=np.array(y_train_data)
x_train_data = Scaler.fit_transform(x_train_data)

x_train_data=torch.FloatTensor(x_train_data)
y_train_data=torch.FloatTensor(y_train_data)      # y데이터가 FloatTensor 형이다.

In [ ]:
train_dataset = torch.utils.data.TensorDataset(x_train_data, y_train_data)

In [ ]:
# Tensor 데이터셋을 통해 DataLoader를 사용할 수 있다.

data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

![대체 텍스트](https://user-images.githubusercontent.com/11758940/83336289-229ec200-a2ed-11ea-9fb8-88a51198e475.png)

In [ ]:
#linear1 = torch.nn.Linear(13,1,bias=True)

linear1 = torch.nn.Linear(13,512,bias=True)
linear2 = torch.nn.Linear(512,512,bias=True)
linear3 = torch.nn.Linear(512,1,bias=True)

relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

[메뉴얼] https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_

In [ ]:
# Random Init => Xavier Init
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)


Parameter containing:
tensor([[-2.5232e-02,  7.4012e-02, -2.6983e-02, -5.8732e-03, -6.8674e-02,
          1.0801e-03,  5.5246e-02,  2.1627e-02, -8.1325e-02, -5.7981e-02,
         -6.2981e-02,  8.3793e-02, -2.7099e-02,  3.7306e-02,  1.7982e-02,
         -3.3500e-02,  4.8161e-02, -6.6684e-02, -7.0474e-02,  5.6616e-02,
          2.9783e-02,  9.5706e-02,  5.0730e-02,  7.7860e-03,  6.3860e-02,
          9.6119e-02, -1.1522e-02, -9.4184e-02, -6.0141e-02, -3.4349e-02,
         -5.1398e-02,  3.2665e-02,  4.7062e-02, -4.7963e-02, -5.3838e-02,
         -9.0761e-02, -9.9320e-02, -9.3208e-02, -7.8838e-02, -5.1631e-02,
         -2.6706e-02, -9.2609e-02, -2.0587e-02, -1.0519e-01, -4.5574e-02,
         -4.2306e-02,  9.4669e-02, -9.1892e-02,  5.2686e-02, -2.9779e-02,
          4.8092e-02,  9.9021e-02, -1.0584e-01,  1.0205e-01, -8.8469e-02,
          1.7143e-02, -8.6990e-02,  9.9737e-02, -5.3751e-02,  9.1031e-03,
          7.2964e-02, -1.5162e-02, -2.4623e-02,  1.6055e-02,  8.7991e-02,
         -1.9344

In [ ]:
# ======================================
# relu는 맨 마지막 레이어에서 빼는 것이 좋다.
# ======================================

#model = torch.nn.Sequential(linear1).to(device)

model = torch.nn.Sequential(linear1, relu, dropout, 
                            linear2, relu, dropout,
                            linear3).to(device)

[메뉴얼] https://pytorch.org/docs/stable/nn.html#crossentropyloss

## regressor는 손실함수로 MSELoss를 사용한다.

- 해당 손실함수로 배추값예측과 같은 선형회귀문제를 풀 수 있다.
- 분류문제에서는 softmax를 내부적으로 계산하는 CrossEntropyLoss()를 사용했다.

In [ ]:
# 손실함수와 최적화 함수
loss = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0001 cost = 80362.015625000
Epoch: 0002 cost = 757.757751465
Epoch: 0003 cost = 636.956909180
Epoch: 0004 cost = 624.437255859
Epoch: 0005 cost = 639.381958008
Epoch: 0006 cost = 626.188354492
Epoch: 0007 cost = 641.902832031
Epoch: 0008 cost = 614.595520020
Epoch: 0009 cost = 617.188842773
Epoch: 0010 cost = 625.122253418
Epoch: 0011 cost = 602.054565430
Epoch: 0012 cost = 585.727539062
Epoch: 0013 cost = 590.479187012
Epoch: 0014 cost = 607.359497070
Epoch: 0015 cost = 599.881103516
Epoch: 0016 cost = 576.344482422
Epoch: 0017 cost = 582.410217285
Epoch: 0018 cost = 580.715332031
Epoch: 0019 cost = 588.816772461
Epoch: 0020 cost = 582.211608887
Epoch: 0021 cost = 572.876220703
Epoch: 0022 cost = 550.628417969
Epoch: 0023 cost = 547.061645508
Epoch: 0024 cost = 540.962890625
Epoch: 0025 cost = 560.615417480
Epoch: 0026 cost = 550.895629883
Epoch: 0027 cost = 553.982788086
Epoch: 0028 cost = 533.446960449
Epoch: 0029 cost = 539.882141113
Epoch: 0030 cost = 514.668945312
Epoch: 0



### test 셋도 Scaler를 통해 transofrom을 해준다.



1.   train 셋의 transfrom:   **fit을 사용**
- Scaler.fit_transform(x_train_data)
- fit -> train 데이터셋을 통해서 어떤 방식으로 scaler를 할지 계산을 구한다는 의미.

2. test셋의 transform: **그냥 transform만**
-  Scaler.transform(x_test_data) 
- fit_transform(train)을 통해 구해진 scaler 방식을 test셋에 적용하겠다는 의미. 

In [ ]:
# Test the model using test sets
with torch.no_grad():

  x_test_data=test_data.loc[:,:]
  x_test_data=np.array(x_test_data)
  x_test_data = Scaler.transform(x_test_data)         # test 셋도 Scaler를 통해 transofrom을 해준다.      
  x_test_data=torch.from_numpy(x_test_data).float().to(device)

  prediction = model(x_test_data)

# argmax를 하지 않는다. predication자체가 예측값이 되기 때문에.

In [ ]:
correct_prediction = prediction.cpu().numpy().reshape(-1,1)

In [ ]:
submit=pd.read_csv('sample_submission.csv')
submit

,id,Price
0,0.0,1.0
1,1.0,1.0
2,2.0,1.0
3,3.0,1.0
4,4.0,1.0
...,...,...
162,162.0,1.0
163,163.0,1.0
164,164.0,1.0
165,165.0,1.0


In [ ]:
for i in range(len(correct_prediction)):
  submit['Price'][i]=correct_prediction[i].item()

submit

,id,Price
0,0.0,22.429939
1,1.0,22.429939
2,2.0,22.429939
3,3.0,22.429939
4,4.0,22.429939
...,...,...
162,162.0,22.429939
163,163.0,22.429939
164,164.0,22.429939
165,165.0,22.429939


In [ ]:
submit.to_csv('baseline.csv',index=False,header=True)

!kaggle competitions submit -c 2020aiboston -f baseline.csv -m "3layer dropout, 13>512>1"

100% 3.82k/3.82k [00:10<00:00, 377B/s]
Successfully submitted to 2020.AI.Boston

### Evaluation: RMSE
- 해당 regression문제의 Evaluation은 RMSE 이다. (분류문제는 accuracy를 사용)
- 0에 가까운 값이 이상적인 값(답)이다.